In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
import os

HOST_IP = os.environ['HOST_IP']
DATABASE_USER = os.environ['DATABASE_USER']
DATABASE_PASSWORD = os.environ['DATABASE_PASSWORD']
DATABASE_PORT = os.environ['DATABASE_PORT']

connection_url = URL.create(
    "postgresql+psycopg2",
    username=DATABASE_USER,
    password=DATABASE_PASSWORD,
    host=HOST_IP,
    port=DATABASE_PORT,
    database="mimiciv"
)

engine = create_engine(connection_url)

In [ ]:
import pandas as pd
from sqlalchemy import text

query = text("SELECT * FROM mimiciv.mimiciv_hosp.admissions LIMIT 10")
pd.read_sql_query(query, engine)

In [ ]:
query = text("""
SELECT json_build_object(
    schema_name, json_agg(
        json_build_object(
            table_name, column_names
        )
    )
)
FROM (
    SELECT 
        t.table_schema as schema_name, 
        t.table_name, 
        json_agg(c.column_name ORDER BY c.ordinal_position) as column_names
    FROM information_schema.tables t
    INNER JOIN information_schema.columns c 
        ON t.table_schema = c.table_schema AND t.table_name = c.table_name
    WHERE t.table_schema NOT IN ('information_schema', 'pg_catalog')
    GROUP BY t.table_schema, t.table_name
) AS sub
GROUP BY schema_name;
""")

# Execute the query
with engine.connect() as con:
    result = con.execute(query).fetchall()

In [ ]:
# Extract schemas and their respective tables with columns from the query result
schemas_with_tables = [schema_result[0] for schema_result in result]

# Flatten the list of dictionaries for each schema
database_structure = {}
for schema in schemas_with_tables:
    for schema_name, tables in schema.items():
        # Initialize the schema in the flattened dictionary if not already present
        if schema_name not in database_structure:
            database_structure[schema_name] = {}

        # Combine the tables under the same schema
        for table in tables:
            database_structure[schema_name].update(table)

database_structure

In [ ]:
def to_clean_records(dataframe):
    return dataframe.apply(lambda row: row.dropna().to_dict(), axis=1).tolist()

In [ ]:
# subject_id = 10000032
hadm_id = 29079034
# hadm_id = 29642964

query = text("select * from mimiciv.mimiciv_hosp.admissions where hadm_id = :hadm;").bindparams(hadm=hadm_id)

hospital_stay = pd.read_sql_query(query, engine)

subject_id = int(hospital_stay['subject_id'][0])

hosp_tables_to_ignore = [
    'admissions',
    'drgcodes',
    'emar',
    'hcpcsevents',
    'pharmacy'
]

hosp_cols_to_remove = {
    'main': ['subject_id', 'hadm_id', 'admit_provider_id', 'insurance', 'edregtime', 'edouttime', 'hospital_expire_flag'],
    'diagnoses_icd': ['subject_id', 'hadm_id', 'seq_num'],
    'labevents': ['subject_id', 'hadm_id', 'labevent_id', 'order_provider_id', 'storetime', 'specimen_id'],
    'microbiologyevents': ['subject_id', 'hadm_id', 'order_provider_id', 'storetime', 'microevent_id', 'micro_specimen_id', 'spec_itemid', 'test_seq', 'test_itemid', 'chartdate'],
    'poe': ['subject_id', 'hadm_id', 'order_provider_id', 'order_status', 'poe_seq'],
    'prescriptions': ['subject_id', 'hadm_id', 'pharmacy_id', 'order_provider_id', 'gsn', 'ndc', 'formulary_drug_cd', 'poe_seq'],
    'procedures_icd': ['subject_id', 'hadm_id', 'seq_num'],
    'services': ['subject_id', 'hadm_id'],
    'transfers': ['subject_id', 'hadm_id', 'transfer_id'],
    'omr': ['subject_id', 'seq_num'],
    'patients': ['subject_id', 'anchor_year_group']
}

for column, v in database_structure['mimiciv_hosp'].items():
    if 'hadm_id' in v:
        if column in hosp_tables_to_ignore:
            continue

        print(column)
        sql_query = text(f"select * from mimiciv.mimiciv_hosp.{column} where hadm_id = :hadm_id").bindparams(hadm_id=hadm_id)
        sql_df = pd.read_sql(sql_query, engine)            
        
        removed_cols = hosp_cols_to_remove[column]
        sql_df = sql_df.drop(removed_cols, axis=1, errors='ignore')

        # if isinstance(sql_df, pd.DataFrame):
        #     sql_df = to_clean_records(sql_df)

        hospital_stay[column] = [sql_df]
        
hospital_stay = hospital_stay.drop(
    hosp_cols_to_remove['main'], axis=1, errors='ignore'
)

In [ ]:
# additional tables from subject id
additional_patient_tables = [
    'omr',
    'patients'
]

for table in additional_patient_tables:
    print(table)
    sql_query = text(f"select * from mimiciv.mimiciv_hosp.{table} where subject_id = :subject_id").bindparams(subject_id=subject_id)
    sql_df = pd.read_sql(sql_query, engine)
    removed_cols = hosp_cols_to_remove[table]
    sql_df = sql_df.drop(removed_cols, axis=1, errors='ignore')

    # if isinstance(sql_df, pd.DataFrame):
    #     sql_df = to_clean_records(sql_df)

    hospital_stay[table] = [sql_df]

In [ ]:
def convert_icd_to_text(icd_list, icd_type):
    # Prepare a CASE statement for ordering
    case_statement = "CASE "
    for index, item in icd_list.iterrows():
        code = item['icd_code'].strip()  # Remove leading/trailing spaces
        version = item['icd_version']
        case_statement += f"WHEN icd_code = '{code}' AND icd_version = {version} THEN {index} "

    case_statement += "END"

    # Create a WHERE IN clause
    icd_conditions = ", ".join([f"('{item[1]['icd_code'].strip()}', {item[1]['icd_version']})" for item in icd_list.iterrows()])
    sql_query = f"""
    SELECT long_title 
    FROM mimiciv.mimiciv_hosp.d_icd_{icd_type} 
    WHERE (icd_code, icd_version) IN ({icd_conditions})
    ORDER BY {case_statement};
    """

    # Execute the query
    return pd.read_sql(sql_query, engine)['long_title'].tolist()

def convert_lab_id_to_info(labs):
    # Prepare a CASE statement for ordering
    case_statement = "CASE "
    for index, item in labs.iterrows():
        id = item['itemid']
        case_statement += f"WHEN itemid = {id} THEN {index} "

    case_statement += "END"

    # Create a WHERE IN clause
    lab_conditions = ", ".join([str(item[1]['itemid']) for item in labs.iterrows()])
    sql_query = f"""
    SELECT *
    FROM mimiciv.mimiciv_hosp.d_labitems
    WHERE itemid IN ({lab_conditions})
    ORDER BY {case_statement};
    """

    # Execute the query
    returned = pd.read_sql(sql_query, engine)
    
    return labs.merge(returned, on='itemid', how='outer').drop('itemid', axis=1)

In [ ]:
# post processing

for column in hospital_stay.columns:
    print(column)
    match column:
        case 'diagnoses_icd':
            if len(hospital_stay[column][0]) > 0:
                diagnoses_df = hospital_stay[column][0]
                ordered_diagnoses = convert_icd_to_text(diagnoses_df, 'diagnoses')
                
                hospital_stay[column] = [ordered_diagnoses]
        case 'labevents':
            converted = convert_lab_id_to_info(hospital_stay[column][0]).sort_values(by=['charttime'])
            
            converted = converted.rename(columns={
                'valuenum': 'numerical value',
                'valueuom': 'unit of measure',
                'ref_range_lower': 'normal lower end',
                'ref_range_upper': 'normal upper end',
            })
            
            hospital_stay[column] = [converted]
        case 'procedures_icd':
            if len(hospital_stay[column][0]) > 0:
                procedures_df = hospital_stay[column][0]
                ordered_procedures = convert_icd_to_text(procedures_df, 'procedures')
                
                
                procedures_df = procedures_df.drop(['icd_code', 'icd_version'], axis=1).sort_values(by=['chartdate'])
                hospital_stay[column] = [procedures_df]
        case 'prescriptions':
            poe_df = hospital_stay['poe'][0]
            prescriptions_df = hospital_stay['prescriptions'][0]
        
            # Get all the prescriptions that have a non-null value in the `poe_id` column
            poe_prescriptions = prescriptions_df[prescriptions_df['poe_id'].notnull()]
            poe_prescriptions = poe_prescriptions.merge(poe_df, on='poe_id', how='outer')
            
            # if a row in poe_prescriptions has both a starttime and an ordertime, set ordertime as null
            poe_prescriptions.loc[(poe_prescriptions['starttime'].notnull()) & (poe_prescriptions['ordertime'].notnull()), 'ordertime'] = None
            
            poe_prescriptions['temp'] = poe_prescriptions['starttime'].combine_first(poe_prescriptions['ordertime'])
            
            poe_prescriptions = poe_prescriptions.sort_values(by=['temp'])
            poe_prescriptions = poe_prescriptions.drop('temp', axis=1)
                        
            poe_prescriptions = poe_prescriptions.rename(columns={
                'order_type': 'order type',
                'transaction_type': 'transaction type',
                'order_subtype': 'order subtype',
                'drug_type': 'drug type',
                'prod_strength': 'strength',
                'dose_val_rx': 'dose',
                'dose_unit_rx': 'dose unit',
                'form_val_disp': 'amount of medication',
                'form_unit_disp': 'amount unit',
                'route': 'route of administration',
                'doses_per_24_hrs': 'doses per 24 hours',
                'discontinue_of_poe_id': 'discontinues poe id',
                'discontinued_by_poe_id': 'discontinued by poe id',
                'starttime': 'start time',
                'stoptime': 'stop time',
            })
        
            hospital_stay['poe'] = [poe_prescriptions]
        
            hospital_stay = hospital_stay.drop('prescriptions', axis=1)
        case 'poe':
            poe_df = hospital_stay['poe'][0]
            poe_df = poe_df[(poe_df['order_type'] != 'ADT orders') & (poe_df['order_type'] != 'Lab')]
            
            # for each poe_id, check if there is a corresponding row in mimiciv_hosp.poe_detail (sql)
            # if there is, add the poe_detail to the poe row
            poe_ids = poe_df['poe_id'].tolist()
            poe_ids = [str(poe_id) for poe_id in poe_ids]
            
            poe_detail_query = text(f"select * from mimiciv.mimiciv_hosp.poe_detail where poe_id in {tuple(poe_ids)}")
            poe_detail_df = pd.read_sql(poe_detail_query, engine).drop(['poe_seq', 'subject_id'], axis=1)
            
            if len(poe_detail_df) > 0:
                poe_df = poe_df.merge(poe_detail_df, on='poe_id', how='outer')
                
                for i, row in poe_df.iterrows():
                    poe_df.at[i, row['field_name']] = row['field_value']
                
                poe_df = poe_df.drop(['field_name', 'field_value'], axis=1)
                            
            hospital_stay['poe'] = [poe_df]
        case 'microbiologyevents':
            micro_df = hospital_stay[column][0]
            micro_df = micro_df.rename(columns={
                'spec_type_desc': 'specimen type',
                'test_name': 'test name',

            })
            hospital_stay[column] = [micro_df]
        case 'omr':
            omr_df = hospital_stay[column][0]

            omr_df = omr_df.rename(columns={
                'result_name': 'name',
                'result_value': 'value',
            })

            hospital_stay[column] = [omr_df]
        case 'services':
            services_df = hospital_stay[column][0]
            services_df = services_df.rename(columns={
                'curr_service': 'current service',
            })
            hospital_stay[column] = [services_df]
        case 'services':
            transfer_df = hospital_stay[column][0]
            transfer_df = transfer_df.drop('transfer_id', axis=1)
            transfer_df = transfer_df.rename(columns={
                'eventtype': 'event type',
                'careunit': 'care unit',
                'intime': 'in time',
                'outtime': 'out time',
            })

            hospital_stay[column] = [transfer_df]
        case 'patients':
            patients_df = hospital_stay[column][0]
            patients_df = patients_df.rename(columns={
                'anchor_age': 'age at anchor year',
                'anchor_year': 'anchor year',
                'dod': 'date of death',
            })

            hospital_stay[column] = [patients_df]
        case 'race' | 'marital_status' | 'language':
            patients_df = hospital_stay['patients'][0].copy()
            patients_df[column] = hospital_stay[column]
            hospital_stay = hospital_stay.drop(column, axis=1)
            hospital_stay['patients'] = [patients_df]

In [ ]:
# rename columns to be more readable
hospital_stay = hospital_stay.rename(columns={
    'admittime': 'admission time',
    'dischtime': 'discharge time',
    'deathtime': 'death time',
    'admission_type': 'admission type',
    'admission_location': 'admission location',
    'discharge_location': 'discharge location',
    'marital_status': 'marital status',
    'diagnoses_icd': 'diagnoses',
    'labevents': 'lab tests',
    'microbiologyevents': 'microbiology tests',
    'poe': 'provider orders',
    'procedures_icd': 'procedures',
    'services': 'hospital services',
    'omr': 'other patient information',
    'patients': 'patient information'
})

In [ ]:
col_order = [
    'patient information',
    'race', 'marital status', 'language',
    'admission time', 'admission type', 'admission location',
    'hospital services', 'transfers',
    'lab tests', 'microbiology tests', 'procedures',
    'provider orders',
    'other patient information',
    'discharge time', 'discharge location', 'death time',
    'diagnoses'
]

hospital_stay = hospital_stay[
    [col for col in col_order if col in hospital_stay.columns]
]

In [ ]:
hospital_stay

In [ ]:
ed_stay_query = text("select * from mimiciv.mimiciv_ed.edstays where hadm_id = :hadm_id").bindparams(hadm_id=hadm_id)

# stay_id = 32522732
# ed_stay_query = text("select * from mimiciv.mimiciv_ed.edstays where stay_id = :stay_id").bindparams(stay_id=stay_id)

ed_stay_df = pd.read_sql(ed_stay_query, engine)

ed_stays = []

ed_cols_to_remove = {
    'main': ['subject_id', 'race', 'gender', 'stay_id'],
    'diagnosis': ['subject_id', 'stay_id', 'seq_num', 'icd_code', 'icd_version'],
    'medrecon': ['subject_id', 'stay_id', 'gsn', 'ndc', 'etc_rn', 'etccode', 'gsn'],
    'pyxis': ['subject_id', 'stay_id', 'gsn', 'med_rn', 'gsn', 'gsn_rn'],
    'triage': ['subject_id', 'stay_id'],
    'vitalsign': ['subject_id', 'stay_id']
}


for row in ed_stay_df.iterrows():
    row_info = pd.DataFrame(row[1]).transpose()
    stay_id = row_info['stay_id'].values[0]

    for column, value in database_structure['mimiciv_ed'].items():
        if 'stay_id' in value:
            if column == 'edstays':
                continue

            print(column)
            sql_query = text(f"select * from mimiciv.mimiciv_ed.{column} where stay_id = :stay_id").bindparams(
                stay_id=stay_id)
            sql_df = pd.read_sql(sql_query, engine)
            
            removed_cols = ed_cols_to_remove[column]
            sql_df = sql_df.drop(removed_cols, axis=1, errors='ignore')
            
            match column:
                case 'diagnosis':
                    # convert to a list of icd_title column
                    sql_df = sql_df['icd_title'].tolist()
                case 'medrecon':
                    sql_df = sql_df.drop('charttime', axis=1, errors='ignore')
                    sql_df = sql_df.rename(columns={'etcdescription': 'enhanced therapeutic class'})
                case 'pyxis':
                    # remove duplicate rows if `name` AND `charttime` are the same
                    sql_df = sql_df.drop_duplicates(subset=['name', 'charttime']) 
            
            # if isinstance(sql_df, pd.DataFrame):
            #     sql_df = to_clean_records(sql_df)

            row_info[column] = [sql_df]
    ed_stays.append(row_info)

ed_stays = pd.concat(ed_stays, ignore_index=True)

ed_stays = ed_stays.drop(
    ed_cols_to_remove['main'], axis=1, errors='ignore'
)

# rename columns to be more readable
ed_stays = ed_stays.rename(columns={
    'intime': 'arrival time',
    'outtime': 'exit time',
    'arrival_transport': 'arrival transport',
    'medrecon': 'medication reconciliation',
    'pyxis': 'dispensed medications',
    'vitalsign': 'vital signs'
})


# reorder columns
ed_stays = ed_stays[[
    'arrival time', 'arrival transport',
    'triage',
    'vital signs',
    'medication reconciliation',
    'dispensed medications',
    'exit time', 'disposition',
    'diagnosis'
]]

In [ ]:
ed_stays

In [ ]:
discharge_note_query = text("select * from mimiciv.mimiciv_note.discharge where hadm_id = :hadm_id").bindparams(
    hadm_id=hadm_id)

discharge_note_df = pd.read_sql(discharge_note_query, engine)

discharge_note = discharge_note_df['text'][0]

hospital_stay['discharge note'] = [discharge_note]

discharge_note

In [ ]:
hospital_stay

In [ ]:
ed_stays

In [ ]:
timeline = []

# add hospital stay
timeline.append({
    'type': 'Hospital Stay',
    'data': hospital_stay,
    'time': hospital_stay['admission time'][0]
})

# add ed stays
for i, row in ed_stays.iterrows():
    timeline.append({
        'type': 'Emergency Department Stay',
        'data': pd.DataFrame([row]),
        'time': row['arrival time']
    })

timeline

In [ ]:
# sort timeline by time
timeline = sorted(timeline, key=lambda k: k['time'])

timeline

In [ ]:
radiology_note_query = text("select * from mimiciv.mimiciv_note.radiology where subject_id = :subject_id and charttime < :discharge_time").bindparams(
    subject_id=subject_id, discharge_time=timeline[-1]['time'])

radiology_note_df = pd.read_sql(radiology_note_query, engine)

# iterate through each note_id
# for each note_id, get the radiology_detail
radiology_note_details = pd.read_sql(text("select * from mimiciv.mimiciv_note.radiology_detail where note_id in :note_id").bindparams(note_id=tuple(radiology_note_df['note_id'])), engine)

radiology_details_pivot = radiology_note_details.pivot_table(
    index=['note_id', 'subject_id'],
    columns='field_name',
    values='field_value',
    aggfunc='first'
).reset_index()

radiology_note_df = pd.merge(
    radiology_note_df,
    radiology_details_pivot,
    on=['note_id', 'subject_id'],
    how='left'
).sort_values(by=['charttime'])

radiology_note_df = radiology_note_df.drop(
    ['subject_id', 'hadm_id', 'note_seq', 'storetime'],
    axis=1
)

# rename note_type based on a mapping
# RR = Radiology Report
# AR = Radiology Report Addendum
radiology_note_df['note_type'] = radiology_note_df['note_type'].map({
    'RR': 'Radiology Report',
    'AR': 'Radiology Report Addendum'
})

column_name_mapping = {
    'note_id': 'note id',
    'note_type': 'type',
    'charttime': 'chart time',
    'parent_note_id': 'parent note id',
    'cpt_code': 'cpt code',
    'exam_code': 'exam code',
    'addendum_note_id': 'addendum note id',
    'exam_name': 'exam name',
}

# rename columns to be more readable
radiology_note_df = radiology_note_df.rename(columns=column_name_mapping)

hospital_stay['radiology notes'] = [radiology_note_df]

radiology_note_df

In [ ]:
def format_df_to_text(df):
    output = ""

    for row in df.iterrows():
        for column in df.columns:
            if column in ['subject_id', 'hadm_id']:
                continue

            value = row[1][column]
            
            if isinstance(value, pd.DataFrame):
                value = to_clean_records(value)

            if isinstance(value, list) and len(value) > 0:
                output += f"{column}:\n"

                if isinstance(value[0], dict):
                    for i, dictionary in enumerate(value):
                        output += f" - {i + 1} of {len(value)}\n"
                        for k, v in dictionary.items():
                            if str(v).lower() not in ['none', 'nan', 'nat']:
                                output += f"   - {k}: {v}\n"
                else:
                    for v in value:
                        if str(v).lower() not in ['none', 'nan', 'nat']:
                            output += f" - {v}\n"
            elif not isinstance(value, list) and str(value).lower() not in ['none', 'nan', 'nat']:
                output += f"{column}: {value}\n"
                
        output += "\n"

    return output


def patient_timeline_to_text(patient_preamble, timeline, hospital_diagnosis):
    patient_info = "Patient Information:\n"
    
    patient_info += format_df_to_text(patient_preamble) + "\n"
    
    for i, event in enumerate(timeline):
        if event['type'] == 'Hospital Stay':
            event['data'] = event['data'].drop(['patient information', 'diagnoses'], axis=1, errors='ignore')
        
        patient_info += f"{event['type']} ({i + 1} of {len(timeline)})\n"
        patient_info += format_df_to_text(event['data'])
        patient_info += "\n"
        
    patient_info += "Diagnoses:\n"
    for diagnosis in hospital_diagnosis:
        patient_info += f" - {diagnosis}\n"
        
    return patient_info
    

In [ ]:
patient_info = hospital_stay['patient information'][0]
hospital_diagnosis = hospital_stay['diagnoses'][0]

patient_timeline_to_text(patient_info, timeline, hospital_diagnosis)